<a href="https://colab.research.google.com/github/AnnaM-C/computational-intelligence/blob/main/COM3013LAB8%269PSO_wip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import torch
import torch.nn.functional as F
from numpy import genfromtxt
import matplotlib.pyplot as plt
import numpy as np
import operator
import random
import numpy
import math
!pip install deap
from deap import base
from deap import benchmarks
from deap import creator
from deap import tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 3.4 MB/s eta 0:00:00


In [2]:
# load the training data
data = genfromtxt('cancer_TR.dat', delimiter=' ')
x = data[:, 0:9]
y = data[:, 9:11]
x = torch.as_tensor(x, dtype=torch.float32)
y = torch.as_tensor(y, dtype=torch.float32)

In [3]:
print(x.size())
print(y)


torch.Size([525, 9])
tensor([[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [0., 1.],
        [1., 0.],
        [1., 0.]])


In [4]:
# torch.manual_seed(1)    # reproducible experiments
                          # by fixing the seed you will remove randomness
# set up the network
class Net(torch.nn.Module):
    # initialise one hidden layer and one output layer
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()
        self.hidden = torch.nn.Linear(n_feature, n_hidden)  # hidden layer
        self.out = torch.nn.Linear(n_hidden, n_output)  # output layer

    # connect up the layers: the input passes through the hidden, then the sigmoid, then the output layer
    def forward(self, x):
        x = F.relu(self.hidden(x))  # activation function for hidden layer
        x = self.out(x)
        return x

In [5]:
net = Net(n_feature=9, n_hidden=5, n_output=2)  # define the network
print(net)  # net architecture

Net(
  (hidden): Linear(in_features=9, out_features=5, bias=True)
  (out): Linear(in_features=5, out_features=2, bias=True)
)


In [15]:
posMinInit      = -3
posMaxInit      = + 5
VMaxInit        = 1.5
VMinInit        = 0.5
populationSize  = 50
dimension       = 20
interval        = 10
#TODO: CHANGE THIS BACK TO 400
iterations      = 4

#Parameter setup

wmax = 0.9 #weighting
wmin = 0.4
c1   = 2.0
c2   = 2.0

creator.create("FitnessMin", base.Fitness, weights=(-1.0,)) # -1 is for minimise
creator.create("Particle", list, fitness=creator.FitnessMin, speed=list, smin=None, smax=None, best=None)
# particle rerpresented by list of 5 things
# 1. fitness of the particle,
# 2. speed of the particle which is also going to be a list,
# 3.4. limit of the speed value,
# 5. best state the particle has been in so far.

def generate(size, smin, smax):
    part = creator.Particle(random.uniform(posMinInit, posMaxInit) for _ in range(size))
    part.speed = [random.uniform(VMinInit, VMaxInit) for _ in range(size)]
    part.smin = smin #speed clamping values
    part.smax = smax
    return part



def updateParticle(part, best, weight):
    #implementing speed = 0.7*(weight*speed + c1*r1*(localBestPos-currentPos) + c2*r2*(globalBestPos-currentPos))
    #Note that part and part.speed are both lists of size dimension
    #hence all multiplies need to apply across lists, so using e.g. map(operator.mul, ...

    r1 = (random.uniform(0, 1) for _ in range(len(part)))
    r2 = (random.uniform(0, 1) for _ in range(len(part)))

    v_r0 = [weight*x for x in part.speed]
    v_r1 = [c1*x for x in map(operator.mul, r1, map(operator.sub, part.best, part))] # local best
    v_r2 = [c2*x for x in map(operator.mul, r2, map(operator.sub, best, part))] # global best

    part.speed = [0.7*x for x in map(operator.add, v_r0, map(operator.add, v_r1, v_r2))]

#not using clamping
#     #clamp limits
#     for i, speed in enumerate(part.speed):
#         if abs(speed) < part.smin:
#             part.speed[i] = math.copysign(part.smin, speed)
#         elif abs(speed) > part.smax:
#             part.speed[i] = math.copysign(part.smax, speed)

    # update position with speed
    part[:] = list(map(operator.add, part, part.speed))

toolbox = base.Toolbox()
toolbox.register("particle", generate, size=dimension, smin=-3, smax=3)
toolbox.register("population", tools.initRepeat, list, toolbox.particle)
toolbox.register("update", updateParticle)
toolbox.register("evaluate", benchmarks.sphere) #sphere function is built-in in DEAP

In [10]:
'''Define Training Variables'''
# optimizer = torch.optim.Rprop(net.parameters(), lr=0.02)
    # SGD or Rprop
# loss_func = torch.nn.MSELoss()
running_loss = 0.0
loss_values = []

In [11]:
'''Test'''
# load the test data
test = genfromtxt('cancer_tt.dat', delimiter=' ')
x2 = test[:, 0:9]
y2 = test[:, 9:11]
x2 = torch.as_tensor(x, dtype=torch.float32)
y2 = torch.as_tensor(y, dtype=torch.float32)

In [12]:
_, target_indices = torch.max(y2, 1)
print(target_indices)
total = y2.size(0)
accuracy = 0

tensor([0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
        1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1,
        1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1,
        1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0,
        0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
        0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0,
        1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1,
        1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0,
        0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
        1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,

In [16]:
for rep in range(10):
    '''Train'''
    net.hidden.reset_parameters()
    net.out.reset_parameters()
#     for layer in net.children():
#         if hasattr(layer, 'reset_parameters'):
#             layer.reset_parameters()
    for t in range(60):
        out = net(x)  # input x and predict based on x

############################ GRADIENT OPTIMISER ################################
        # loss = loss_func(out, y)
        # optimizer.zero_grad()  # clear gradients for next train
        # loss.backward()  # backpropagation, compute gradients
        # optimizer.step()  # apply gradients
############################ GRADIENT OPTIMISER ################################

############################.      PSO.        .################################
        pop = toolbox.population(n=populationSize) # Population Size
        stats = tools.Statistics(lambda ind: ind.fitness.values)
        stats.register("avg", numpy.mean)
        stats.register("std", numpy.std)
        stats.register("min", numpy.min)
        stats.register("max", numpy.max)

        logbook = tools.Logbook()
        logbook.header = ["gen", "evals"] + stats.fields

        best = None

        #begin main loop
        for g in range(iterations):
            w = wmax - (wmax-wmin)*g/iterations #decaying inertia weight

            for part in pop:
                part.fitness.values = toolbox.evaluate(part) #actually only one fitness value

                #update local best
                if (not part.best) or (part.best.fitness < part.fitness):   #lower fitness is better (minimising)
                #   best is None   or  current value is better              #< is overloaded
                    part.best = creator.Particle(part)
                    part.best.fitness.values = part.fitness.values

                #update global best
                if (not best) or best.fitness < part.fitness:
                    best = creator.Particle(part)
                    best.fitness.values = part.fitness.values

            for part in pop:
                toolbox.update(part, best,w)

            # Gather all the fitnesses in one list and print the stats
            # print every interval
            if g%interval==0: # interval
                logbook.record(gen=g, evals=len(pop), **stats.compile(pop))
                print(logbook.stream)
                #print('best ',best, best.fitness)

        print('best particle position is ', best)

        # TODO:
        # Put the best particles weights into the network
        print(best)





Streaming output truncated to the last 5000 lines.
200	50   	0.000357986	2.98424e-09	0.000357983	0.000357996
210	50   	0.000357983	4.22367e-11	0.000357983	0.000357983
220	50   	0.000357983	5.93783e-12	0.000357983	0.000357983
230	50   	0.000357983	3.49393e-12	0.000357983	0.000357983
240	50   	0.000357983	5.19642e-12	0.000357983	0.000357983
250	50   	0.000357983	4.43825e-12	0.000357983	0.000357983
260	50   	0.000357983	8.8311e-12 	0.000357983	0.000357983
270	50   	0.000357982	9.92354e-12	0.000357982	0.000357982
280	50   	0.000357982	1.15713e-11	0.000357982	0.000357982
290	50   	0.000357982	9.86444e-12	0.000357982	0.000357982
300	50   	0.000357982	9.52385e-12	0.000357982	0.000357982
310	50   	0.000357982	6.77996e-12	0.000357982	0.000357982
320	50   	0.000357981	9.28901e-12	0.000357981	0.000357981
330	50   	0.000357981	6.137e-12  	0.000357981	0.000357981
340	50   	0.000357981	5.40812e-12	0.000357981	0.000357981
350	50   	0.000357981	5.21955e-12	0.000357981	0.000357981
360	50   	0.000357981

KeyboardInterrupt: ignored

In [ ]:
        loss_values.append(loss.item()) # keep track of loss values for later plot

        _, predicted_indices = torch.max(net(x2), 1)
        accuracythistime = (predicted_indices == target_indices ).sum().item() / total
        print("accuracy ",rep,":", accuracythistime)

        accuracy += accuracythistime

        accuracy/=10
############################.      PSO.        .################################


print("accuracy:", accuracy)